In [28]:
import mysql.connector
import pandas as pd
import string
import random
import csv
from random import randint
import time
import contextlib
import matplotlib.pyplot as plt

from ipywidgets import widgets as w
from IPython.display import display

In [29]:
NFL_teams=('Arizona Cardinals', 'Atlanta Falcons', 'Baltimore Ravens', 'Buffalo Bills', 'Carolina Panthers', 'Chicago Bears',
'Cincinnati Bengals', 'Cleveland Browns', 'Dallas Cowboys', 'Denver Broncos', 'Detroit Lions', 'Green Bay Packers',
'Houston Texans','Indianapolis Colts', 'Jacksonville Jaguars', 'Kansas City Chiefs', 'Miami Dolphins', 'Minnesota Vikings',
'New England Patriots', 'New Orleans Saints', 'New York Giants', 'New York Jets', 'Oakland Raiders', 'Philadelphia Eagles',
'Pittsburgh Steelers', 'St. Louis Rams', 'San Diego Chargers', 'San Francisco 49ers', 'Seattle Seahawks','Tampa Bay Buccaneers',
'Tennessee Titans','Washington Redskins')

In [30]:
NFL_stadiums=('Arrowhead', 'AT&T', 'Bank of America', 'Broncos', 'CenturyLink Field', 'FedExField', 'FirstEnergy',
             'Ford Field', 'Gillette', 'Hard Rock', 'Heinz Field', 'Lambeau Field')

In [31]:
NFL_city=('Arizona', 'Atlanta', 'Baltimore', 'Buffalo Bills', 'Carolina', 'Chicago',
'Cincinnati', 'Cleveland', 'Dallas', 'Denver', 'Detroit','Houston','Indianapolis',
'Jacksonville', 'Kansas', 'Miami', 'Minnesota','New England Patriots', 'New Orleans Saints',
'New York Giants', 'New York Jets', 'Oakland Raiders', 'Philadelphia Eagles',
'Pittsburgh','San Diego', 'San Francisco', 'Seattle','Tampa Bay','Tennessee','Washington')

In [32]:
def random_date():
    year = randint(2000, 2019)
    month = randint(1, 12)
    if(month<10):
        month='0'+str(month)
    day = randint(1, 28)
    if(day<10):
        day='0'+str(day)
    return str(year)+str(month)+str(day)

In [33]:
def get_TeamIDs():
    database_conector = mysql.connector.connect(user='root', password='usa220231609',
                                 host='127.0.0.1', database='nfl',
                                 auth_plugin='mysql_native_password')
    pointer = database_conector.cursor()
    pointer.execute("SELECT TeamID FROM teams")
    result = pointer.fetchall()
    return [item[0] for item in result]

In [34]:
def get_PlayerIDs():
    db = mysql.connector.connect(user='root', password='usa220231609',
                                 host='127.0.0.1', database='nfl',
                                 auth_plugin='mysql_native_password')
    cursor = db.cursor()
    cursor.execute("SELECT PlayerID FROM Players")
    result = cursor.fetchall()
    return [item[0] for item in result]

In [35]:
def get_GameIDs():
    db = mysql.connector.connect(user='root', password='usa220231609',
                                 host='127.0.0.1', database='nfl',
                                 auth_plugin='mysql_native_password')
    cursor = db.cursor()
    cursor.execute("SELECT GameID FROM Games")
    result = cursor.fetchall()
    #return result
    return [item[0] for item in result]

# Generate Data Functions

In [36]:
# Generate Data Functions
Team_IDs = []
Player_IDs = []
Game_IDs = []
Play_IDs = []
def generate_teams_data(numberOfRows, output_filename='teams.csv'):
    """creates a csv file with numberOfRows number of rows for the teams table
    teams Table:
    TeamID     int,
    Team_name    varchar(20) not null,
    city         varchar(20) not null,
    primary key (TeamID)
    """
    global Team_IDs
    Team_IDs = [] # Reset for each time we generate data
    
    ID_set=set() # storing the ID's since primary key should be unique.
    list_of_rows=[]
    with open(output_filename, mode='w', newline='') as player_file:
        writer = csv.writer(player_file, delimiter=',')
        
        for _ in range(numberOfRows):
            TeamID = randint(1, 1000000000)
            while(TeamID in ID_set):
                TeamID = randint(1, 1000000000)
            ID_set.add(TeamID)
            Team_IDs.append(TeamID)
            Team_name = random.choice(NFL_teams)
            city = random.choice(NFL_city)
            
            list_of_rows.append([TeamID, Team_name, city])
        writer.writerows(list_of_rows)

        

def generate_players_data(numberOfRows, output_filename='players.csv'):
    """creates a csv file with numberOfRows number of rows for the player table
    Player Table: 
    PlayerID     int,
    TeamID     int,
    FirstName:   varchar(20) not null,
    LastName:    varchar(20) not null,
    Position     varchar(2)  check (Position in ('QB','RB','WR')),
    Touchdowns   int check   (Touchdowns > 0),
    TotalYards  int check   (TotalYards > 0),
    Salary       int check   (Salary > 0 ),
    primary key (PlayerID)
    foreign key (TeamID) references teams (TeamID)
    """
    global Player_IDs
    Players_IDs = [] # Reset for each time we generate data
    
    ID_set=set() # storing the ID's since primary key should be unique.
    list_of_rows=[]        
    with open(output_filename, mode='w', newline='') as player_file:
        writer = csv.writer(player_file, delimiter=',')
        
        for _ in range(numberOfRows):
            
            # Create random PlayerID
            PlayerID = randint(1, 1000000000)
            
            # Get random TeamID from Team_IDs list
            TeamID = random.choice(Team_IDs)
            
            while(PlayerID in ID_set):
                PlayerID = randint(1, 1000000000)
                TeamID = random.choice(Team_IDs)
            ID_set.add(PlayerID)
            Player_IDs.append(PlayerID)
        
            FirstName = ''.join(random.choice('abcdefghijklmnopqrstuvwxyz') for _ in range(5, 10))
            LastName = ''.join(random.choice('abcdefghijklmnopqrstuvwxyz') for _ in range(5, 10))
            Position = random.choice(('QB','RB','WR'))
            Touchdowns = randint(1,1000)
            TotalYards = randint(1, 1000000)
            Salary = randint(1000, 1000000)
            
            list_of_rows.append([PlayerID, TeamID, FirstName, LastName, Position, Touchdowns, TotalYards, Salary])
        writer.writerows(list_of_rows)
        
def generate_games_data(numberOfRows, output_filename='games.csv'):
    """creates a csv file with numberOfRows number of rows for the game table
    Game Table:
    GameID         int,
    Date           date,
    Stadium        varchar(20),
    Result         char check (Result in ('W','L','T')),
    Attendance     int check (Attendance > 0),
    TicketRevenue int check (TicketRevenue > 0),
    primary key (GameID)
    """
    global Game_IDs
    Game_IDs = [] # Reset for each time we generate data
    
    ID_set=set()
    list_of_rows=[]
    with open(output_filename, mode='w', newline='') as game_file:
        writer = csv.writer(game_file, delimiter=',')
        
        for _ in range(numberOfRows):
            GameID = randint(1, 1000000)
            while(GameID in ID_set):
                GameID = randint(1, 1000000)
            ID_set.add(GameID)
            Game_IDs.append(GameID)
            Date = random_date()
            Stadium = random.choice(NFL_stadiums)
            Result = random.choice(('W','L','T'))
            Attendance = randint(1, 1000000)
            TicketRevenue = randint(1, 1000000000)
            
            list_of_rows.append([GameID, Date, Stadium, Result, Attendance, TicketRevenue])
        writer.writerows(list_of_rows)
        
def generate_play_data(numberOfRows, output_filename='play.csv'):
    """creates a csv file with numberOfRows number of rows for the play table
    Play Table:
    PlayerID       int,
    GameID         int,
    primary key (PlayerID, GameID),
    foreign key (PlayerID) references Players(PlayerID),
    foreign key (GameID) references Games(GameID),
    """
    global Play_IDs
    Play_IDs = [] # Reset for each time we generate data
    
    ID_set=set()
    list_of_rows=[]
    with open(output_filename, mode='w', newline='') as play_file:
        writer = csv.writer(play_file, delimiter=',')

        
        for _ in range(numberOfRows):
            PlayerID = random.choice(Player_IDs)
            GameID = random.choice(Game_IDs)
            while((PlayerID, GameID) in ID_set):
                PlayerID = random.choice(Player_IDs)
                GameID = random.choice(Game_IDs)
            ID_set.add((PlayerID, GameID))
            Play_IDs.append((PlayerID, GameID))
            
            list_of_rows.append([PlayerID, GameID])
        writer.writerows(list_of_rows)

# Generate Data

## Size 10,000

In [10]:
generate_teams_data(10000 ,'teams10000.csv')

In [11]:
generate_players_data(10000,'players10000.csv')

In [12]:
generate_games_data(10000, 'games10000.csv')

In [13]:
generate_play_data(10000,'play10000.csv')

## Size 100,000

In [ ]:
generate_teams_data(100000,'teams100000.csv')

In [ ]:
generate_players_data(100000,'players100000.csv')

In [ ]:
generate_games_data(100000, 'games100000.csv')

In [ ]:
generate_play_data(100000,'play100000.csv')

## Size 1,000,000

In [ ]:
generate_teams_data(1000000,'teams1000000.csv')

In [ ]:
generate_players_data(1000000,'players1000000.csv')

In [ ]:
generate_games_data(1000000, 'games1000000.csv')

In [ ]:
generate_play_data(1000000,'play1000000.csv')

# Connect to SQL

In [37]:
database_connector = mysql.connector.connect(user='root', password='usa220231609',
                                host='127.0.0.1', database='nfl',
                                auth_plugin='mysql_native_password', allow_local_infile=True)
pointer = database_connector.cursor()

# Helper Delete Table Functions

In [38]:
# Delete Tables
def delete_players():
    sql = "DELETE FROM players"
    pointer.execute(sql)
    database_connector.commit()
    print("Delete complete")
    
def delete_teams():
    sql = "DELETE FROM teams"
    pointer.execute(sql)
    database_connector.commit()
    print("Delete complete")
    
def delete_games():
    sql = "DELETE FROM Games"
    pointer.execute(sql)
    database_connector.commit()
    print("Delete complete")
    
def delete_play():
    sql = "DELETE FROM Play"
    pointer.execute(sql)
    database_connector.commit()
    print("Delete complete")

# Helper Insert into table functions

In [66]:
# Insert into Tables
def insert_teams(file_name='teams.csv'):
    vals=[]
    with open(file_name) as input_file:
        print("Inserting: ", file_name)
        reader = csv.reader(input_file, delimiter=',')
        for row in reader:
            vals.append(tuple(row))

        start = time.time()
        for row in vals:
            sql = "INSERT INTO teams (TeamID, TeamName, city) VALUES (%s, %s, %s)"
            try:
                pointer.execute(sql, row)
            except mysql.connector.Error as e:
                return e.msg
        database_connector.commit()
        end = time.time()
        print(end - start, 'seconds')
        s=str(end - start) + 'seconds'
        return s
    
def insert_players(file_name='players.csv'):
    vals=[]
    with open(file_name) as input_file:
        print("Inserting: ", file_name)
        reader = csv.reader(input_file, delimiter=',')
        for row in reader:
            vals.append(tuple(row))

        start = time.time()
        for row in vals:
            sql = "INSERT INTO players (PlayerID, TeamID, FirstName, LastName, Position, Touchdowns, TotalYards, Salary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            try:
                pointer.execute(sql, row)
            except mysql.connector.Error as e:
                return e.msg
        database_connector.commit()
        end = time.time()
        print(end - start, 'seconds')
        s=str(end - start) + 'seconds'
        return s
    
def insert_games(file_name='games.csv'):
    vals=[]
    with open(file_name) as input_file:
        print("Inserting: ", file_name)
        reader = csv.reader(input_file, delimiter=',')
        for row in reader:
            vals.append(tuple(row))

        start = time.time()
        for row in vals:
            sql = "INSERT INTO Games (GameID, Date, Stadium, Result, Attendance, TicketRevenue) VALUES (%s, %s, %s, %s, %s, %s)"
            try:
                pointer.execute(sql, row)
            except mysql.connector.Error as e:
                print(e.msg)
                return e.msg
        database_connector.commit()
        end = time.time()
        print(end - start, 'seconds')
        s=str(end - start) + 'seconds'
        return s

def insert_play(file_name='plays.csv'):
    vals=[]
    with open(file_name) as input_file:
        print("Inserting: ", file_name)
        reader = csv.reader(input_file, delimiter=',')
        for row in reader:
            vals.append(tuple(row))

        start = time.time()
        for row in vals:
            sql = "INSERT INTO Play (PlayerID, GameID) VALUES (%s, %s)"
            try:
                pointer.execute(sql, row)
            except mysql.connector.Error as e:
                print(e.msg)
                return e.msg
        database_connector.commit()
        end = time.time()
        print(end - start, 'seconds')
        s=str(end - start) + 'seconds'
        return s

# Retrieve table functions

In [85]:
def retrieve_teams():
    sql = "SELECT * FROM TEAMS"
    pointer.execute(sql)
    
    out_retrieve.clear_output()
    with out_retrieve:
        print("Team ID, Team Name, City")
        for a in pointer:
            print(a)

def retrieve_players():
    sql = "SELECT * FROM PLAYERS"
    pointer.execute(sql)
    
    out_retrieve.clear_output()
    with out_retrieve:
        print("Player ID, Team ID, First Name, Last Name, Position, Touchdowns, Total Yards, Salary")
        for a in pointer:
            print(a)


def retrieve_games():
    sql = "SELECT * FROM GAMES"
    pointer.execute(sql)
    
    out_retrieve.clear_output()
    with out_retrieve:
        print("Game ID, Date, Stadium, Result, Attendance, Ticket Revenue")
        for a in pointer:
            print(a)

def retrieve_play():
    sql = "SELECT * FROM PLAY"
    pointer.execute(sql)
    
    out_retrieve.clear_output()
    with out_retrieve:
        print("Player ID, Game ID")
        for a in pointer:
            print(a)

# Retrieve Average Function

In [95]:
def retrieve_average(table, column):
    sql = "SELECT AVG(" + column + ") FROM " + table
    pointer.execute(sql)
    out_average.clear_output()
    with out_average:
        print("Average of ", table, " and ", column, ": ")
        for a in pointer:
            print(a)

# Create Widgets
## Output

In [87]:
out_insert = w.Output(layout={'border': '1px solid black'})
out_delete = w.Output(layout={'border': '1px solid black'})
out_retrieve = w.Output(layout={'border': '1px solid black'})
out_average = w.Output(layout={'border': '1px solid black'})

## Table and File

In [88]:
# Tables (remove, retrieve, average)
table = w.Text(description="Table: ")
table_type = w.RadioButtons(
    options=['teams','players', 'games', 'play']
)

# File Upload
file_upload = w.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


## Insert

In [89]:
# Single insert
single_insert = w.Button(description="Single Insert")

def _on_single_insert_clicked(single_insert):
    out_insert.clear_output()
    with out_insert:
        if table_type.value == 'teams':
            insert_teams(file_name=next(iter(file_upload.value)))    
        elif table_type.value == 'players':
            insert_players(file_name=next(iter(file_upload.value)))
        elif table_type.value == 'games':
            insert_games(file_name=next(iter(file_upload.value)))
        elif table_type.value == 'play':
            insert_play(file_name=next(iter(file_upload.value)))
single_insert.on_click(_on_single_insert_clicked)


# Multiple row insert
multiple_row_insert = w.Button(description="Multiple Row Insert")

def _on_multiple_row_insert_clicked(multiple_row_insert):
    
    # clear output widget
    out_insert.clear_output()
    
    with out_insert:
        print("multiple row insert clicked")
        
        # get name of file
        file_name = next(iter(file_upload.value))
        
        # read file into pandas dataframe
        df = pd.read_csv(file_name, sep=',', header=None)
        # clean data
        cols = str(df.columns.values.tolist()).replace('[','').replace(']', '')
        values = list(map(tuple, df.values))
        half = len(values) // 2
        valuesA = values[:half] # get first half
        valuesB = values[half:] # get second half

        # create sql statement
        if table_type.value == 'teams':
            sql = "insert into teams ( TeamID, TeamName, City ) values  (%s, %s, %s )" 
        elif table_type.value == 'players':
            sql = "INSERT INTO players (PlayerID, TeamID, FirstName, LastName, Position, Touchdowns, TotalYards, Salary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        elif table_type.value == 'games':
            sql = "INSERT INTO games (GameID, Date, Stadium, Result, Attendance, TicketRevenue) VALUES (%s, %s, %s, %s, %s, %s)"
        elif table_type.value == 'play':
            sql = "INSERT INTO Play (PlayerID, GameID) VALUES (%s, %s)"
        
        # start time
        start = time.time()

        # execute sql query in 2 halves
        try:
            pointer.executemany(sql, valuesA)
        except mysql.connector.Error as e:
            print(e.msg)
            return e.msg
        database_connector.commit()
        try:
            pointer.executemany(sql, valuesB)
        except mysql.connector.Error as e:
            return e.msg
        database_connector.commit()
        
        # end time
        end = time.time()
        
        # print time
        print(end - start, 'seconds for multi row play')
        s=str(end - start) + 'seconds'
        return s
multiple_row_insert.on_click(_on_multiple_row_insert_clicked)


# Load data syntax
load_data_syntax = w.Button(description="Load Data Syntax")

def _on_load_data_syntax_clicked(load_data_syntax): # ONLY FOR TEAMS
    
    # clear output widget
    out_insert.clear_output()
    
    # with output widget
    with out_insert:
        
        # get name of file
        file_name = next(iter(file_upload.value))
        
        # start time
        start = time.time()
        
        # create sql statement
        if table_type.value == 'teams':
            sql = "LOAD DATA LOCAL INFILE \'%s\' INTO TABLE teams FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\r\\n' (TeamID, TeamName, city)" % (file_name)

        elif table_type.value == 'players':
            sql = "LOAD DATA LOCAL INFILE \'%s\' INTO TABLE players FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\r\\n' (PlayerID, TeamID, FirstName, LastName, Position, Touchdowns, TotalYards, Salary)" % (file_name)

        elif table_type.value == 'games':
            sql = "LOAD DATA LOCAL INFILE \'%s\' INTO TABLE games FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\r\\n' (GameID, Date, Stadium, Result, Attendance, TicketRevenue)" % (file_name)

        elif table_type.value == 'play':
            sql = "LOAD DATA LOCAL INFILE \'%s\' INTO TABLE play FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\r\\n' (PlayerID, GameID)" % (file_name)


        # execute query
        try:
            pointer.execute(sql)
        except mysql.connector.Error as e:
            print(e.msg)
            return e.msg
        
        # commit to database
        database_connector.commit()
        
        # end timer
        end = time.time()
        
        # print time in output widget
        print(end - start, 'seconds')
        s=str(end - start) + 'seconds'
        return s
load_data_syntax.on_click(_on_load_data_syntax_clicked)

## Remove

In [121]:
# Remove a table
remove_table = w.Button(description="Remove Table", button_style='danger')

def _on_remove_table_clicked(remove_table):
    out_delete.clear_output()
    with out_delete:
        if table_type.value == 'teams':
            print("Teams deleting..")
            delete_teams()    
        elif table_type.value == 'players':
            print("Players deleting..")
            delete_players()
        elif table_type.value == 'games':
            print("Games deleting..")
            delete_games()
        elif table_type.value == 'play':
            print("Play deleting..")
            delete_play()
remove_table.on_click(_on_remove_table_clicked)



# Remove ALL tables
remove_all_tables = w.Button(description="Remove ALL Tables", button_style='danger')

def _on_remove_all_tables_clicked(remove_all_tables):
    out_delete.clear_output()
    with out_delete:
        print("Play deleting..")
        delete_play()
        print("Games deleting..")
        delete_games()
        print("Players deleting..")
        delete_players()
        print("Teams deleting..")
        delete_teams()
remove_all_tables.on_click(_on_remove_all_tables_clicked)


## Retrieve

In [124]:
# Retrieve a table
retrieve_table = w.Button(description="Retrieve Table")

def _on_retrieve_table_clicked(retrieve_table):
    out_retrieve.clear_output()
    with out_retrieve:
        if table_type.value == 'teams':
            print("Retrieving teams table..")
            retrieve_teams()    
        elif table_type.value == 'players':
            print("Retrieving players table..")
            retrieve_players()
        elif table_type.value == 'games':
            print("Retrieving games table..")
            retrieve_games()
        elif table_type.value == 'play':
            print("Retrieving play table..")
            retrieve_play()
retrieve_table.on_click(_on_retrieve_table_clicked)

## Average

In [125]:
# Get Average

column = w.Text(description ="Column: ")
average_table_column = w.Button(description="Find Average")

def _on_average_table_column_clicked(average_table_column):
    out_average.clear_output()
    with out_average:
        retrieve_average(table_type.value, column.value)
        
average_table_column.on_click(_on_average_table_column_clicked)

# Create User Interface

In [129]:
upload = w.VBox([
    w.Label(value="Select a text file:"),
    file_upload,
    w.Label(value="Select table type: "),
    table_type,
    w.Label(value="Use one of the following methods to upload data:"),
    single_insert,
    multiple_row_insert,
    load_data_syntax,
    out_insert,
])

left_delete = w.VBox([
    w.Label(value="Select table type: "),
    table_type, 
    remove_table,
    out_delete,
])
right_delete = w.VBox([
    w.Label(value="Want to delete all tables? "),
    remove_all_tables,
])

delete = w.HBox([
    left_delete,
    right_delete,
])


retrieve = w.VBox([
    w.Label(value="Select table type: "),
    table_type, 
    retrieve_table,   
    out_retrieve,
])

average = w.VBox([
    w.Label(value="Select table type: "),
    table_type, 
    column,
    average_table_column,
    out_average,
])

user_interface = w.Accordion(children=[upload, delete, retrieve, average])
user_interface.set_title(0, 'upload')
user_interface.set_title(1, 'delete')
user_interface.set_title(2, 'retrieve')
user_interface.set_title(3, 'average')


# User Interface

In [131]:
display(user_interface)
out_insert.clear_output()
out_delete.clear_output()
out_retrieve.clear_output()
out_average.clear_output()

Accordion(children=(VBox(children=(Label(value='Select a text file:'), FileUpload(value={'teams10000.csv': {'m…